In [1]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from urllib import request
from scipy.interpolate import interp1d
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import speclite.filters as sp
from speclite import filters
from scipy.ndimage import sum_labels, mean
from scipy.stats import linregress, spearmanr


from astropy.io import fits
from astropy import units as u
from astropy import constants as c
from astropy.wcs import WCS
from astropy.wcs.utils import proj_plane_pixel_scales

from ppxf.ppxf import ppxf, rebin
import ppxf.ppxf_util as util
from ppxf import sps_util as lib

import os
import sys
import glob

In [2]:
# Load gas line map NGC4298_gas_BIN_maps.fits 
gas_path = Path('NGC4298_gas_BIN_maps_extended.fits')
print(f"Loading gas line map from {gas_path}")
with fits.open(gas_path) as hdul:
    V_STARS2 = hdul['V_STARS2'].data
    SIGMA_STARS2 = hdul['SIGMA_STARS2'].data
    HB4861_FLUX = hdul['HB4861_FLUX'].data
    HB4861_FLUX_ERR = hdul['HB4861_FLUX_ERR'].data
    HA6562_FLUX = hdul['HA6562_FLUX'].data
    HA6562_FLUX_ERR = hdul['HA6562_FLUX_ERR'].data
    OIII5006_FLUX = hdul['OIII5006_FLUX'].data
    OIII5006_FLUX_ERR = hdul['OIII5006_FLUX_ERR'].data
    NII6583_FLUX = hdul['NII6583_FLUX'].data
    NII6583_FLUX_ERR = hdul['NII6583_FLUX_ERR'].data
    SII6716_FLUX = hdul['SII6716_FLUX'].data
    SII6716_FLUX_ERR = hdul['SII6716_FLUX_ERR'].data
    SII6730_FLUX = hdul['SII6730_FLUX'].data
    SII6730_FLUX_ERR = hdul['SII6730_FLUX_ERR'].data
    gas_header = hdul[5].header
    hdul.close()

gas_header

Loading gas line map from NGC4298_gas_BIN_maps_extended.fits


XTENSION= 'IMAGE   '           / Image extension                                
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 1042                                                  
NAXIS2  =                  995                                                  
PCOUNT  =                    0 / number of parameters                           
GCOUNT  =                    1 / number of groups                               
WCSAXES =                    2 / Number of coordinate axes                      
CRPIX1  =      216.11231877588 / Pixel coordinate of reference point            
CRPIX2  =      435.42596878359 / Pixel coordinate of reference point            
CDELT1  = -5.5555555555556E-05 / [deg] Coordinate increment at reference point  
CDELT2  =  5.5555555555556E-05 / [deg] Coordinate increment at reference point  
CUNIT1  = 'deg'             

In [3]:
# Apply a first cut of FLUX/ERR ≥ 5 on every line, 
# at least 22 to get min BD>2.86
# at least 15 to have all SF in BPT diagram
cut = 0
mask_HB = HB4861_FLUX / HB4861_FLUX_ERR >= cut
mask_HA = HA6562_FLUX / HA6562_FLUX_ERR >= cut
# Combine masks for both lines
mask_combined = mask_HB & mask_HA
# Apply the mask to the flux maps
HB4861_FLUX_cut = np.where(mask_combined, HB4861_FLUX, np.nan)
HA6562_FLUX_cut = np.where(mask_combined, HA6562_FLUX, np.nan)

# Balmer-decrement map – H α/H β (start with all spaxels).
BD = HA6562_FLUX_cut / HB4861_FLUX_cut
# ---- line ratios --------------------------------------------------
logN2  = np.log10(NII6583_FLUX / HA6562_FLUX)        # [N II]/Hα
logS2  = np.log10((SII6716_FLUX+SII6730_FLUX) / HA6562_FLUX)   # Σ[S II]/Hα
logO3  = np.log10(OIII5006_FLUX / HB4861_FLUX)       # [O III]/Hβ         

mask_N2 = NII6583_FLUX / NII6583_FLUX_ERR >= cut
mask_S2 = (SII6716_FLUX + SII6730_FLUX) / (SII6716_FLUX_ERR + SII6730_FLUX_ERR) >= cut
mask_O3 = OIII5006_FLUX / OIII5006_FLUX_ERR >= cut
mask_combinedd = mask_combined & mask_N2 & mask_S2 & mask_O3

logN2_cut = np.where(mask_combinedd, logN2, np.nan)
logS2_cut = np.where(mask_combinedd, logS2, np.nan)
logO3_cut = np.where(mask_combinedd, logO3, np.nan)

/var/folders/rm/clp89v210v32vh5vjt2kx7b80000gr/T/ipykernel_15248/2764862445.py:14: RuntimeWarning: divide by zero encountered in divide
  BD = HA6562_FLUX_cut / HB4861_FLUX_cut
/var/folders/rm/clp89v210v32vh5vjt2kx7b80000gr/T/ipykernel_15248/2764862445.py:14: RuntimeWarning: invalid value encountered in divide
  BD = HA6562_FLUX_cut / HB4861_FLUX_cut
/var/folders/rm/clp89v210v32vh5vjt2kx7b80000gr/T/ipykernel_15248/2764862445.py:16: RuntimeWarning: divide by zero encountered in divide
  logN2  = np.log10(NII6583_FLUX / HA6562_FLUX)        # [N II]/Hα
/var/folders/rm/clp89v210v32vh5vjt2kx7b80000gr/T/ipykernel_15248/2764862445.py:16: RuntimeWarning: invalid value encountered in divide
  logN2  = np.log10(NII6583_FLUX / HA6562_FLUX)        # [N II]/Hα
/var/folders/rm/clp89v210v32vh5vjt2kx7b80000gr/T/ipykernel_15248/2764862445.py:16: RuntimeWarning: divide by zero encountered in log10
  logN2  = np.log10(NII6583_FLUX / HA6562_FLUX)        # [N II]/Hα
/var/folders/rm/clp89v210v32vh5vjt2kx7b8

In [4]:
#  N II BPT -----------------------------------------
def kewley01_N2(x):   # max-starburst
    return 0.61/(x-0.47) + 1.19
def kauff03_N2(x):    # empirical SF upper envelope
    return 0.61/(x-0.05) + 1.30                            

#  S II BPT -----------------------------------------
def kewley01_S2(x):
    return 0.72/(x-0.32) + 1.30                           
def kewley06_Sy_LIN(x):   # Seyfert/LINER division
    return 1.89*x + 0.76        

# Calculate flux-weighted galaxy representative points
# Assuming you have a flux array (replace 'flux' with your actual flux variable name)
# If you don't have flux, you can use Ha_flux or another appropriate flux measurement
valid_mask_N2 = np.isfinite(logN2_cut) & np.isfinite(logO3_cut) 
valid_mask_S2 = np.isfinite(logS2_cut) & np.isfinite(logO3_cut) 

# Calculate flux-weighted representative points
galaxy_logN2 = np.log10(np.nansum(10**logN2_cut[valid_mask_N2] * 10**logN2_cut[valid_mask_N2]) / np.nansum(10**logN2_cut[valid_mask_N2]))
galaxy_logO3_N2 = np.log10(np.nansum(10**logO3_cut[valid_mask_N2] * 10**logO3_cut[valid_mask_N2]) / np.nansum(10**logO3_cut[valid_mask_N2]))
galaxy_logS2 = np.log10(np.nansum(10**logS2_cut[valid_mask_S2] * 10**logS2_cut[valid_mask_S2]) / np.nansum(10**logS2_cut[valid_mask_S2]))
galaxy_logO3_S2 = np.log10(np.nansum(10**logO3_cut[valid_mask_S2] * 10**logO3_cut[valid_mask_S2]) / np.nansum(10**logO3_cut[valid_mask_S2]))

# Print galaxy representative points
print(f"Galaxy flux-weighted representative points:")
print(f"[N II] BPT: logN2 = {galaxy_logN2:.3f}, logO3 = {galaxy_logO3_N2:.3f}")
print(f"[S II] BPT: logS2 = {galaxy_logS2:.3f}, logO3 = {galaxy_logO3_S2:.3f}")

# Count the number of spaxels in each region
N2_HII = logO3 <= kauff03_N2(logN2)
N2_Comp = (logO3 > kauff03_N2(logN2)) & (logO3 <= kewley01_N2(logN2))
N2_AGN = logO3 > kewley01_N2(logN2)
S2_HII = logO3 <= kewley01_S2(logS2)
S2_Seyfert = (logO3 > kewley01_S2(logS2)) & (logO3 > kewley06_Sy_LIN(logS2))
S2_LINER = (logO3 > kewley01_S2(logS2)) & (logO3 <= kewley06_Sy_LIN(logS2))
# Count the number of spaxels in each region
N2_HII_count = np.sum(N2_HII)
N2_Comp_count = np.sum(N2_Comp)
N2_AGN_count = np.sum(N2_AGN)
S2_HII_count = np.sum(S2_HII)
S2_Seyfert_count = np.sum(S2_Seyfert)
S2_LINER_count = np.sum(S2_LINER)
print(f"Number of spaxels in [N II] BPT regions:")
print(f"HII: {N2_HII_count}, Comp: {N2_Comp_count}, AGN: {N2_AGN_count}")
print(f"Number of spaxels in [S II] BPT regions:")
print(f"HII: {S2_HII_count}, Seyfert: {S2_Seyfert_count}, LINER: {S2_LINER_count}")

Galaxy flux-weighted representative points:
[N II] BPT: logN2 = 0.003, logO3 = 1.583
[S II] BPT: logS2 = -0.189, logO3 = 1.192
Number of spaxels in [N II] BPT regions:
HII: 182075, Comp: 112546, AGN: 61438
Number of spaxels in [S II] BPT regions:
HII: 235646, Seyfert: 26197, LINER: 85916


In [5]:
# Purely SF spaxels in both BPT diagram
mask_SF = (N2_HII+N2_Comp) & (S2_HII) #& (E_BV_BD > -0.5)  
# Apply the mask to the Halpha flux map
HA6562_FLUX_SF = np.where(mask_SF, HA6562_FLUX_cut, np.nan)
HB4861_FLUX_SF = np.where(mask_SF, HB4861_FLUX_cut, np.nan)
OIII5006_FLUX_SF = np.where(mask_SF, OIII5006_FLUX, np.nan)
NII6583_FLUX_SF = np.where(mask_SF, NII6583_FLUX, np.nan)
SII6716_FLUX_SF = np.where(mask_SF, SII6716_FLUX, np.nan)
SII6730_FLUX_SF = np.where(mask_SF, SII6730_FLUX, np.nan)
HA6562_FLUX_ERR_SF = np.where(mask_SF, HA6562_FLUX_ERR, np.nan)
HB4861_FLUX_ERR_SF = np.where(mask_SF, HB4861_FLUX_ERR, np.nan)
OIII5006_FLUX_ERR_SF = np.where(mask_SF, OIII5006_FLUX_ERR, np.nan)
NII6583_FLUX_ERR_SF = np.where(mask_SF, NII6583_FLUX_ERR, np.nan)
SII6716_FLUX_ERR_SF = np.where(mask_SF, SII6716_FLUX_ERR, np.nan)
SII6730_FLUX_ERR_SF = np.where(mask_SF, SII6730_FLUX_ERR, np.nan)
BD_SF = np.where(mask_SF, BD, np.nan)

In [6]:
HA6562_FLUX_SNR = HA6562_FLUX/HA6562_FLUX_ERR
HB4861_FLUX_SNR = HB4861_FLUX/HB4861_FLUX_ERR
OIII5006_FLUX_SNR = OIII5006_FLUX/OIII5006_FLUX_ERR
NII6583_FLUX_SNR = NII6583_FLUX/NII6583_FLUX_ERR
SII6716_FLUX_SNR = SII6716_FLUX/SII6716_FLUX_ERR
SII6730_FLUX_SNR = SII6730_FLUX/SII6730_FLUX_ERR

threshold = 2.5  # BD threshold
good = BD_SF >= threshold
between = good & (BD_SF < 2.86)

In [7]:
BD_SF_good = np.where(good, BD_SF, np.nan)
BD_SF_bad = np.where(~good, BD_SF, np.nan)
BD_SF_between = np.where(between, BD_SF, np.nan)

In [8]:
HA6562_FLUX_SF_good = np.where(good, HA6562_FLUX_SF, np.nan)
HB4861_FLUX_SF_good = np.where(good, HB4861_FLUX_SF, np.nan)
OIII5006_FLUX_SF_good = np.where(good, OIII5006_FLUX_SF, np.nan)
NII6583_FLUX_SF_good = np.where(good, NII6583_FLUX_SF, np.nan)
SII6716_FLUX_SF_good = np.where(good, SII6716_FLUX_SF, np.nan)
SII6730_FLUX_SF_good = np.where(good, SII6730_FLUX_SF, np.nan)
HA6562_FLUX_SF_bad = np.where(~good, HA6562_FLUX_SF, np.nan)
HB4861_FLUX_SF_bad = np.where(~good, HB4861_FLUX_SF, np.nan)
OIII5006_FLUX_SF_bad = np.where(~good, OIII5006_FLUX_SF, np.nan)
NII6583_FLUX_SF_bad = np.where(~good, NII6583_FLUX_SF, np.nan)
SII6716_FLUX_SF_bad = np.where(~good, SII6716_FLUX_SF, np.nan)
SII6730_FLUX_SF_bad = np.where(~good, SII6730_FLUX_SF, np.nan)
HA6562_FLUX_SF_between = np.where(between, HA6562_FLUX_SF, np.nan)
HB4861_FLUX_SF_between = np.where(between, HB4861_FLUX_SF, np.nan)
OIII5006_FLUX_SF_between = np.where(between, OIII5006_FLUX_SF, np.nan)
NII6583_FLUX_SF_between = np.where(between, NII6583_FLUX_SF, np.nan)
SII6716_FLUX_SF_between = np.where(between, SII6716_FLUX_SF, np.nan)
SII6730_FLUX_SF_between = np.where(between, SII6730_FLUX_SF, np.nan)
HA6562_FLUX_SF_poor = np.where(~good | between, HA6562_FLUX_SF, np.nan)
HB4861_FLUX_SF_poor = np.where(~good | between, HB4861_FLUX_SF, np.nan)
OIII5006_FLUX_SF_poor = np.where(~good | between, OIII5006_FLUX_SF, np.nan)
NII6583_FLUX_SF_poor = np.where(~good | between, NII6583_FLUX_SF, np.nan)
SII6716_FLUX_SF_poor = np.where(~good | between, SII6716_FLUX_SF, np.nan)
SII6730_FLUX_SF_poor = np.where(~good | between, SII6730_FLUX_SF, np.nan)

HA6562_FLUX_ERR_SF_good = np.where(good, HA6562_FLUX_ERR_SF, np.nan)
HB4861_FLUX_ERR_SF_good = np.where(good, HB4861_FLUX_ERR_SF, np.nan)
OIII5006_FLUX_ERR_SF_good = np.where(good, OIII5006_FLUX_ERR_SF, np.nan)
NII6583_FLUX_ERR_SF_good = np.where(good, NII6583_FLUX_ERR_SF, np.nan)
SII6716_FLUX_ERR_SF_good = np.where(good, SII6716_FLUX_ERR_SF, np.nan)
SII6730_FLUX_ERR_SF_good = np.where(good, SII6730_FLUX_ERR_SF, np.nan)
HA6562_FLUX_ERR_SF_bad = np.where(~good, HA6562_FLUX_ERR_SF, np.nan)
HB4861_FLUX_ERR_SF_bad = np.where(~good, HB4861_FLUX_ERR_SF, np.nan)
OIII5006_FLUX_ERR_SF_bad = np.where(~good, OIII5006_FLUX_ERR_SF, np.nan)
NII6583_FLUX_ERR_SF_bad = np.where(~good, NII6583_FLUX_ERR_SF, np.nan)
SII6716_FLUX_ERR_SF_bad = np.where(~good, SII6716_FLUX_ERR_SF, np.nan)
SII6730_FLUX_ERR_SF_bad = np.where(~good, SII6730_FLUX_ERR_SF, np.nan)
HA6562_FLUX_ERR_SF_between = np.where(between, HA6562_FLUX_ERR_SF, np.nan)
HB4861_FLUX_ERR_SF_between = np.where(between, HB4861_FLUX_ERR_SF, np.nan)
OIII5006_FLUX_ERR_SF_between = np.where(between, OIII5006_FLUX_ERR_SF, np.nan)
NII6583_FLUX_ERR_SF_between = np.where(between, NII6583_FLUX_ERR_SF, np.nan)
SII6716_FLUX_ERR_SF_between = np.where(between, SII6716_FLUX_ERR_SF, np.nan)
SII6730_FLUX_ERR_SF_between = np.where(between, SII6730_FLUX_ERR_SF, np.nan)
HA6562_FLUX_ERR_SF_poor = np.where(~good | between, HA6562_FLUX_ERR_SF, np.nan)
HB4861_FLUX_ERR_SF_poor = np.where(~good | between, HB4861_FLUX_ERR_SF, np.nan)
OIII5006_FLUX_ERR_SF_poor = np.where(~good | between, OIII5006_FLUX_ERR_SF, np.nan)
NII6583_FLUX_ERR_SF_poor = np.where(~good | between, NII6583_FLUX_ERR_SF, np.nan)
SII6716_FLUX_ERR_SF_poor = np.where(~good | between, SII6716_FLUX_ERR_SF, np.nan)
SII6730_FLUX_ERR_SF_poor = np.where(~good | between, SII6730_FLUX_ERR_SF, np.nan)

In [9]:
# O'Donnell (1994) update
# k_Hb = 3.609   # 4861 Å
# k_Ha = 2.535   # 6563 Å
# Calzetti (2000)
k_Hb = 4.598 
k_Ha = 3.325
R_int = 2.86

E_BV_BD = 2.5/(k_Hb - k_Ha) * np.log10( (HA6562_FLUX_SF_good/HB4861_FLUX_SF_good) / R_int )

In [10]:
# Manually set the negative E(B-V) values to zero
E_BV_BD[E_BV_BD < 0] = 0

In [11]:
# Purely SF spaxels in both BPT diagram
mask_SF = (N2_HII+N2_Comp) & (S2_HII) #& (E_BV_BD > -0.5)  
# Apply the mask to the Halpha flux map
HA6562_FLUX_SF = np.where(mask_SF, HA6562_FLUX_cut, np.nan) 

In [12]:
# Corrected Halpha map with E(B-V) from gas lines
HA6562_FLUX_SF_good_corr = HA6562_FLUX_SF_good * 10**(0.4 * k_Ha * E_BV_BD)

In [13]:
# Convert the corrected Halpha map ($10^{-20}erg/(s cm^2)$) to luminosity (erg/s)
def flux_to_luminosity(flux, distance=16.5):
    """
    Convert flux to luminosity.
    
    Parameters:
    flux : array-like
        Flux in erg/(s * Angstrom * cm^2).
    distance : float
        Distance in parsecs.
        
    Returns:
    luminosity : array-like
        Luminosity in erg/s.
    """
    return (flux*1e-20*u.erg/u.s/u.cm**2 * 4*np.pi*(distance*u.Mpc)**2).cgs

# Convert the corrected Halpha flux map to luminosity
HA6562_LUM_SF_good_corr = flux_to_luminosity(HA6562_FLUX_SF_good_corr)
total_HA6562_LUM_SF_good_corr = np.nansum(HA6562_LUM_SF_good_corr)
# Print the total luminosity
print(f"Total corrected Hα luminosity for purely SF good spaxels: {total_HA6562_LUM_SF_good_corr:.3e} erg/s")


Total corrected Hα luminosity for purely SF good spaxels: 1.418e+41 erg / s erg/s


In [14]:
# SFR map from Halpha luminosity, using Calzetti 2007
def calzetti_sfr(luminosity):
    """
    Convert Halpha luminosity to SFR using Calzetti 2007.
    But it is assuming the Kroupa IMF, 
    so we need to times a coefficient to go to Chabrier IMF.
    
    Parameters:
    luminosity : array-like
        Halpha luminosity in erg/s.
        
    Returns:
    sfr : array-like
        Star formation rate in solar masses per year.
    """
    return 5.3e-42 * luminosity.cgs.value / 0.67 *0.63 # SFR in M_sun/yr

# Calculate SFR map from corrected Halpha luminosity
SFR_map_SF_good_corr = calzetti_sfr(HA6562_LUM_SF_good_corr)
# Calculate total SFR
total_SFR_SF_good_corr = calzetti_sfr(total_HA6562_LUM_SF_good_corr)
print(f"Total SFR for purely SF good spaxels: {total_SFR_SF_good_corr:.3f} M_sun/yr")


Total SFR for purely SF good spaxels: 0.707 M_sun/yr
